In [ ]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [ ]:
from tokenizer import *
tokenizer_bpe = BPETokenizer(corpus, 50)

In [ ]:
special_tokens = ["UNK", "PAD", "CLS", "SEP", "MASK"]
tokenizer_bpe.train(special_tokens)

In [ ]:
tokenizer_bpe.merge_rules

In [ ]:
'\\' in tokenizer_bpe.vocab

In [ ]:
print(tokenizer_bpe.tokenize('This is not a token \\\\\\\.'))

## Count Frequency of Word


In [ ]:
from typing import List, Dict
from tools import *

def count_word_freqs(corpus: List[str])-> Dict[str, int]:
    word_freqs = {}
    word_list = pre_process(corpus)
    for word in word_list:
        if word not in word_freqs.keys():
            word_freqs[word] = 1
        else:
            word_freqs[word] += 1
    return word_freqs

In [ ]:
word_freqs = count_word_freqs(corpus)

In [ ]:
for key in word_freqs.keys():
    print(key, word_freqs[key])

## Compute the base vocabulary

In [ ]:
from typing import Dict, List

# Get the vocabulary 
def get_vocab(word_freqs: Dict[str, int])-> List[str]:
    alphabet = []
    for key in word_freqs.keys():
        for char in key:
            if char not in alphabet:
                alphabet.append(char)
    alphabet.sort()
    base_vocab = ["<|endoftext|>", "UNK"] + alphabet.copy()
    return base_vocab

In [ ]:
vocab = get_vocab(word_freqs)
print(vocab)

In [ ]:
# Get the word as key, and character list as value
splits = {word: [c for c in word] for word in word_freqs.keys()}

In [ ]:
print(splits)

In [ ]:
from collections import defaultdict

# get the pair frequencies based on word_freqs
def compute_pair_freqs(splits, word_freqs):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [ ]:
pair_freqs = compute_pair_freqs(splits, word_freqs)

In [ ]:
pair_freqs

In [ ]:
def merge_pair(a, b, splits, word_freqs):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [ ]:
vocab_size = 50
merges = {}
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits, word_freqs)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits, word_freqs)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [ ]:
print(merges)

In [ ]:
def ps(special_tokens):
    for i in special_tokens:
        print(i)

In [ ]:
k = ['abv', 'z', 'mm']
ps(k)